In [34]:
import av

from PIL import Image

import os
import random
import math

In [35]:
DS_CDFV1 = 'celeb_df_v1/'
DS_CDFV2 = 'celeb_df_v2/'

DS_ORGINAL = 'dataset_original/'
DS_SPLIT = 'dataset_split/'
DS_IFRAMES = 'dataset_iframes/'
DS_FACE = 'dataset_face/'
DS_FACE_IMG = 'dataset_face_img/'
DS_SRM_SNIPPETS = 'dataset_srm_snippets/'
DS_SEGMENTS = 'dataset_segments/'
DS_RAW = 'dataset_raw/'
DS_RESIDUALS = 'dataset_residuals/'

SEG_1 = 'seg_1/'
SEG_2 = 'seg_2/'
SEG_3 = 'seg_3/'
SEG_4 = 'seg_4/'
SEG_5 = 'seg_5/'

DS_TRAIN = 'train_dataset/'
DS_TEST = 'test_dataset/'
DS_VAL = 'val_dataset/'

CLASS_FAKE = 'fake/'
CLASS_REAL = 'real/'


TOP_LEVEL_1 = [DS_SPLIT, DS_IFRAMES, DS_FACE, DS_FACE_IMG, DS_SRM_SNIPPETS]
TOP_LEVEL_2 = [DS_SEGMENTS, DS_RAW, DS_RESIDUALS]
SEGMENTS = [SEG_1, SEG_2, SEG_3, SEG_4, SEG_5]
SPLIT = [DS_TRAIN, DS_TEST, DS_VAL]
CLASS = [CLASS_REAL, CLASS_FAKE]

DATASET = [DS_CDFV1, DS_CDFV2]

# Snippet Extraction

In [25]:
# Returns the index of frames the begin a new segment
def get_segment_dividers(frame_count, num_segments):
    segments_per_frame = math.floor(frame_count / num_segments)

    return [(segments_per_frame * i) for i in range(0, num_segments) ]

In [ ]:
def get_snippet_indices(segment_dividers):
    return []

In [24]:
# Returns an array of randomly selected snippets(PIL.Image) from each segment of the input video
def extract_snippets(fp, num_segments, num_snippets):
    vid_container = av.open(fp)
    vid_stream = vid_container.streams.video[0]

    frame_count = vid_stream.frames
    segment_dividers = get_segment_dividers(frame_count, num_segments)
    snippet_indices = get_snippet_indices(segment_dividers)

    snippets = []

    frame_index = 0
    for frame in vid_container.decode():
        if frame_index > max(snippet_indices):
            break

        if frame_index in snippet_indices:
            snippets.append(frame.to_image())

        frame_index += 1

    return snippets

## Testing Logic

In [33]:
get_segment_dividers(35, 5)

[0, 7, 14, 21, 28]

In [ ]:
test_file = os.listdir(DS_CDFV1 + DS_SPLIT + DS_TRAIN + CLASS_REAL)[0]
test_input = av.open(os.path.realpath(DS_CDFV1 + DS_SPLIT + DS_TRAIN + CLASS_REAL + test_file))

for frame in test_input.decode():
    print(frame.key_frame)

## Implementation

### Celeb-DF v1 & v2

In [ ]:
def save_snippets_CDF(dataset, num_segments, num_snippets):
    if dataset != DS_CDFV1 or dataset != DS_CDFV2:
        return
    
    random.seed(1)
    
    src_base_path = dataset + DS_IFRAMES
    dst_base_path = dataset + DS_SRM_SNIPPETS

    for split in SPLIT:
        for class_dir in CLASS:
            for video in os.listdir(src_base_path + split + class_dir):
                fp = src_base_path + split + class_dir + video
                snippets = extract_snippets(fp, num_segments, num_snippets)

                for i, snippet in enumerate(snippets, start=1):
                    seg_index = math.ceil(float(i) / num_snippets)
                    snip_index = (i - 1) % num_snippets
              
                    dst = f'{dst_base_path + split + class_dir + video}s_{seg_index}_f{snip_index}'
                    snippet.save(dst, format='jpg')

## REMOVE BREAK STATEMENTS AFTER TESTING ##
                break
            break
        break
            

In [ ]:
# CELEB DF V1
save_snippets_CDF(DS_CDFV1, num_segments=5, num_snippets=1)